In [1]:
# Nikhef Modulation Experiment Simulation
# Simulates data of radioactive decay, assuming exponential decay function, with Gaussian statistical uncertainty 
# (intrinsic to radioactive decay) and extra noise from errors in measurement should still be added. 
# A modulation can be manually added.


# Input: 
#    Days_Measured = days measured
#    Num_sample_points = number of sample points, eg: 1 sample per minute => N = 1*60*24*D
#    tau_in_Yr = half life time in years
#    A_0 = activity of radioactive source at t = 0
#    freq, ampl, phaseDg: frequency (1 modulation per day -> 1/(24*60*60)), amplitude and phase of signal (in degrees)
#    sigma_Noise = standard deviation of uncertainty in measurement on signal
#    guess_hLife_time = start value for half life time, set to 1 year, since 60Co, 137Cs and 44Ti have half life times in order of years

# DON'T FORGET TO CHANGE THE #DAYS IN THE NP.SAVETXT-FILENAME!!! 

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import scipy.optimize as optimize
from scipy.optimize import curve_fit
%matplotlib inline

matplotlib.rc('font', size=18)                   # Use big fonts...
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # ... and big plots

#simulation of exponential decay
Days_Measured = 500. # days measured
T = 15*60. #sample spacing [sec] = total time measured (in seconds) /number of sample points
Num_sample_points = (60*60*24.0*Days_Measured)/T # number of sample points (if 1 per minute: 60*24 in 1 day & 1 per 6 min: 10*24 in 1 day)
                                                 # Num_sample_points should preferably be 2^n with n = integer, this to make the FFT work better
day_in_mintes = 24*60 # number of minutes in a day
day_in_seconds = 24*60*60 # number of seconds in a day

tau_in_Yr = 5.27 #half life time in years
tau_in_Sec = tau_in_Yr*365*24*60*60 #half life time in seconds

A0 = 37 #start activity in Bq

x = np.linspace(0, T*Num_sample_points, Num_sample_points)

def exp(x, A_0, half_life_time): # A_0=activity at t=0 in Bq, and half life time in seconds 
    return A_0*np.exp(-x*np.log(2)/half_life_time)

#exponential radioactive decay
def exp_decay(x):
    return exp(x,A0,tau_in_Sec)

In [2]:
# define functions

def mod(x,ampl,freq1,pha):
        return ampl*np.cos(2*np.pi*freq1*x-pha)

# Radioactive decay + modulation:
def exp_decay_and_Modulation(x):
    return exp_decay(x)+mod(x,ampl_input, freq_input,phase_input)

def external_Noise(sigma_noise, x):
    return np.random.normal(0,sigma_noise,len(x))

def exp_decay_mod_all_noise(x, sigma_noise):
    return exp_decay_and_Modulation(x)+external_Noise(sigma_noise, x)

# fit exponent to simulated radioactive decay and find Chi-squared
def fit_signal(signal, guess, x):
    popt, pcov = curve_fit(exp, x, signal, guess) # scipy.optimize.curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=False)
    perr = np.sqrt(np.diag(pcov))
    Fitted_exp_decay = exp(x, *popt)
    return Fitted_exp_decay

# Define the residuals als the data mnis the fitted exponent
def residual(x, exponent_and_modulation_and_noise, fit):
    residual = (exponent_and_modulation_and_noise) - fit
    return residual

   # find modulation frequency using a cosinfe fit with first guess 1 year period
def cosine(y, amplF, freqF, ph):
    return amplF*np.cos(2*np.pi*freqF*y-ph)
def cosineFit(x, residual, guess, dRate):
    (amplitudeFit, freqFit, phaseFit), pcov = optimize.curve_fit(cosine, x, residual, p0=guess, sigma=dRate, absolute_sigma=True)
    perr = np.sqrt(np.diag(pcov))
    return amplitudeFit, freqFit, phaseFit, perr, pcov

# Try to fit a function which is exp+cos, to ensure that the exp-fit is not overestimated due to the modulation
def expANDcos(x,  a0, halfLifeTime, ampli, frequ, phase):
    return exp(x, a0, halfLifeTime)+cosine(x, ampli, frequ, phase)

def expANDcosFit(x, simSignal, guess, dRate):
    (a0Fit, hltFit, amplFit, frequFit, phFit), pcov = optimize.curve_fit(expANDcos, x, simSignal, p0=guess, sigma=dRate, absolute_sigma=True)
    perr = np.sqrt(np.diag(pcov))
    return a0Fit, hltFit, amplFit, frequFit, phFit, perr, pcov


In [9]:
range_ampl = np.arange(0.00, 0.005, 0.0001)
#range_ampl = np.arange(0.005, 0.01, 0.0001)
range_run = np.arange(0., 300, 1.)


ampl_input_array = []
cos_found_freq_varying_ampl_array = []
stdv_cos_found_freq_varying_ampl_array = []
cos_found_ampl_varying_ampl_array = []
stdv_cos_found_ampl_varying_ampl_array = []
cos_found_phase_varying_ampl_array = []
stdv_cos_found_phase_varying_ampl_array = []

expANDcos_found_freq_varying_ampl_array = []
stdv_expANDcos_found_freq_varying_ampl_array = []
expANDcos_found_ampl_varying_ampl_array = []
stdv_expANDcos_found_ampl_varying_ampl_array = []
expANDcos_found_phase_varying_ampl_array = []
stdv_expANDcos_found_phase_varying_ampl_array = []

for ampl_run in range_ampl:

#     # artificially added modulation:
    freq_input = 1./(365.25*24*60*60.0) #frequency of the signal (mod/sec): 1 modulation per day -> 1/(24*60*60)
    ampl_input = ampl_run*A0 #amplitude of the signal, changes in activity of ~0.1% should be detectable
    ampl_input_array.append(ampl_input) 
    phase_in_Degrees = 40.
    phase_input = np.deg2rad(phase_in_Degrees) #phase of the signal

    cos_found_freq_array = []
    cos_found_ampl_array = []
    cos_found_phase_array = []

    std_cos_found_ampl_array = []
    std_cos_found_freq_array = []
    std_cos_found_phase_array = []
    
    expANDcos_found_freq_array = []
    expANDcos_found_ampl_array = []
    expANDcos_found_phase_array = []

    std_expANDcos_found_ampl_array = []
    std_expANDcos_found_freq_array = []
    std_expANDcos_found_phase_array = []

    for run in range_run:
        # Make random Noise
        sigma_Noise_external = np.sqrt(T*exp_decay(x))/(T)  # sigma_A = sigma_N/T = sqrt(N)/T = sqrt(AT)/T
        
        guessA0 = max(exp_decay_mod_all_noise(x, sigma_Noise_external)) #max value is presumably the activity at t=0 (since it only decreases)
        guess_hLife_time = 5.*365*24*60*60.0  #since all sources have a half life time in order of years, start with guess 1 yr
        guessExp = (guessA0, guess_hLife_time)

        first_guess_freq = 1/(365.*24*60*60) # freq guess for cosine fit
        first_guess_cosFit = [0.001, first_guess_freq, 0.5] # total first guess for cosine fit
        
        # determine error on "measured" rate, to use in the curve_fit:
        drate = np.sqrt(T*exp_decay_mod_all_noise(x, sigma_Noise_external))/(T)

        # Find exponential fit of simulates data    
        found_fit_signal = fit_signal(exp_decay_mod_all_noise(x, sigma_Noise_external), guessExp, x)
       
        # Estimate residual
        found_residual = residual(x, exp_decay_mod_all_noise(x, sigma_Noise_external), found_fit_signal)
        
        # Find cosine fit of residual
        found_cosFit = cosineFit(x, found_residual, first_guess_cosFit, drate)
        
        # Find exp+cos fit for simulates data (so not using the residuals)
        guessEC = np.hstack((guessExp, first_guess_cosFit))
        found_expANDcosfit = expANDcosFit(x, exp_decay_mod_all_noise(x, sigma_Noise_external), guessEC, drate)
        
        # Make plots to show expANDcos fit is better than first exp and then cos:
        
#         plt.plot(x/(60*60*24.), expANDcos(x, found_expANDcosfit[0],found_expANDcosfit[1],found_expANDcosfit[2],found_expANDcosfit[3],found_expANDcosfit[4]), label="found_expANDcosfit")
#         plt.xlabel('measured time[days]')
#         plt.ylabel('rate [Hz]')
#         plt.legend(bbox_to_anchor=(1.0, 1), loc=2, borderaxespad=0.)
#         plt.show()
        
#         # Look at found values for the exponent
#         #plt.plot(x/(60*60*24.), exp_decay_mod_all_noise(x, sigma_Noise_external), label="Simulated signal")
#         plt.plot(x/(60*60*24.), found_fit_signal, label="Fitted exponential from exp fit")
#         plt.plot(x/(60*60*24.), exp(x, found_expANDcosfit[0], found_expANDcosfit[1]), label="Fitted exponential with values from expANDcos fit")
#         plt.plot(x/(60*60*24.), exp_decay(x), '--' , label="Input exponential")
#         plt.xlabel('measured time[days]')
#         plt.ylabel('rate [Hz]')
#         plt.legend(bbox_to_anchor=(1.0, 1), loc=2, borderaxespad=0.)
#         plt.show() 
        
#         # Look at found values for the cosine
#         #plt.plot(x/(60*60*24.), found_residual, label="Residual")
#         plt.plot(x/(60*60*24.), [cosine(xi, found_cosFit[0], found_cosFit[1], found_cosFit[2]) for xi in x], label="Fitted cosine from cosfit")
#         plt.plot(x/(60*60*24.), [cosine(xi, found_expANDcosfit[2], found_expANDcosfit[3], found_expANDcosfit[4]) for xi in x], label="Fitted cosine from expANDcosfit")
#         plt.plot(x/(60*60*24.), [cosine(xi, ampl_input, freq_input, phase_input) for xi in x], '--' , label="Input cosine")
#         plt.xlabel('measured time[days]')
#         plt.ylabel('rate [Hz]')
#         plt.legend(bbox_to_anchor=(1.0, 1), loc=2, borderaxespad=0.)
#         plt.show()

        
        #print the found cos frequency, amplitude and phase for the cosfit
        cos_found_ampl_array.append(found_cosFit[0]) 
        cos_found_freq_array.append(found_cosFit[1])
        cos_found_phase_array.append(found_cosFit[2])

        std_cos_found_ampl_array.append(found_cosFit[3][0]) 
        std_cos_found_freq_array.append(found_cosFit[3][1])
        std_cos_found_phase_array.append(found_cosFit[3][2])
        
        #print the found cos frequency, amplitude and phase for the expANDcosfit
        expANDcos_found_ampl_array.append(found_expANDcosfit[2]) 
        expANDcos_found_freq_array.append(found_expANDcosfit[3])
        expANDcos_found_phase_array.append(found_expANDcosfit[4])

        std_expANDcos_found_ampl_array.append(found_expANDcosfit[5][2]) 
        std_expANDcos_found_freq_array.append(found_expANDcosfit[5][3])
        std_expANDcos_found_phase_array.append(found_expANDcosfit[5][4])

    #Make arrays for the cosinefit    
    mean_ampl_cos_array = np.mean(cos_found_ampl_array)
    cos_found_ampl_varying_ampl_array.append(mean_ampl_cos_array)
    stdv_ampl_cos_array = np.mean(std_cos_found_ampl_array)+np.std(cos_found_ampl_array)
    stdv_cos_found_ampl_varying_ampl_array.append(stdv_ampl_cos_array)

    mean_freq_cos_array = np.mean(cos_found_freq_array)
    cos_found_freq_varying_ampl_array.append(mean_freq_cos_array)
    stdv_freq_cos_array = np.mean(std_cos_found_freq_array)+np.std(cos_found_freq_array)
    stdv_cos_found_freq_varying_ampl_array.append(stdv_freq_cos_array)

    mean_phase_cos_array = np.mean(cos_found_phase_array)
    cos_found_phase_varying_ampl_array.append(mean_phase_cos_array)
    stdv_phase_cos_array = np.mean(std_cos_found_phase_array)+np.std(cos_found_phase_array)
    stdv_cos_found_phase_varying_ampl_array.append(stdv_phase_cos_array)
    
    #Make arrays for the expANDcosinefit    
    mean_ampl_expANDcos_array = np.mean(expANDcos_found_ampl_array)
    expANDcos_found_ampl_varying_ampl_array.append(mean_ampl_expANDcos_array)
    stdv_ampl_expANDcos_array = np.mean(std_expANDcos_found_ampl_array)+np.std(expANDcos_found_ampl_array)
    stdv_expANDcos_found_ampl_varying_ampl_array.append(stdv_ampl_expANDcos_array)

    mean_freq_expANDcos_array = np.mean(expANDcos_found_freq_array)
    expANDcos_found_freq_varying_ampl_array.append(mean_freq_expANDcos_array)
    stdv_freq_expANDcos_array = np.mean(std_expANDcos_found_freq_array)+np.std(expANDcos_found_freq_array)
    stdv_expANDcos_found_freq_varying_ampl_array.append(stdv_freq_expANDcos_array)

    mean_phase_expANDcos_array = np.mean(expANDcos_found_phase_array)
    expANDcos_found_phase_varying_ampl_array.append(mean_phase_expANDcos_array)
    stdv_phase_expANDcos_array = np.mean(std_expANDcos_found_phase_array)+np.std(expANDcos_found_phase_array)
    stdv_expANDcos_found_phase_varying_ampl_array.append(stdv_phase_expANDcos_array)

np.savetxt('annual_mod_input_amplitude_CosFit_0.00_0.005_st0.0001', (ampl_input_array))
           
np.savetxt('annual_mod_mean_found_amplitude_CosFit_500_0.005_0.01_300runs_test', (cos_found_ampl_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_amplitude_CosFit_500_0.005_0.01_300runs_test', (stdv_cos_found_ampl_varying_ampl_array))
np.savetxt('annual_mod_mean_found_frequency_CosFit_500_0.005_0.01_300runs_test', (cos_found_freq_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_frequency_CosFit_500_0.005_0.01_300runs_test', (stdv_cos_found_freq_varying_ampl_array))
np.savetxt('annual_mod_mean_found_phase_CosFit_500_0.005_0.01_300runs_test', (cos_found_phase_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_phase_CosFit_500_0.005_0.01_300runs_test', (stdv_cos_found_phase_varying_ampl_array))

np.savetxt('annual_mod_mean_found_amplitude_expANDcosFit_500_0.005_0.01_300runs_test', (expANDcos_found_ampl_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_amplitude_expANDcosFit_500_0.005_0.01_300runs_test', (stdv_expANDcos_found_ampl_varying_ampl_array))
np.savetxt('annual_mod_mean_found_frequency_expANDcosFit_500_0.005_0.01_300runs_test', (expANDcos_found_freq_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_frequency_expANDcosFit_500_0.005_0.01_300runs_test', (stdv_expANDcos_found_freq_varying_ampl_array))
np.savetxt('annual_mod_mean_found_phase_expANDcosFit_500_0.005_0.01_300runs_test', (expANDcos_found_phase_varying_ampl_array))
np.savetxt('annual_mod_stdv_found_phase_expANDcosFit_500_0.005_0.01_300runs_test', (stdv_expANDcos_found_phase_varying_ampl_array))
